In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 11 22:52:51 2017

@author: Tu Bui tb00083@surrey.ac.uk
"""

import sys,os
from PIL import Image
import StringIO
import math
import subprocess
import caffe
import numpy as np
from sklearn.metrics import confusion_matrix
from scipy.io import savemat
import time

MODEL_WEIGHTS_PATH = 'model/triplet1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy_iter_31200.caffemodel'
MODEL_SPEC_PATH = 'model/deploy_images_net1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy.prototxt'



GPU_DEV = 0
LAYER_DIMS=256
mean_pixel = np.array([104, 117, 123],dtype=np.float32)[:,None,None]

def get_net(caffemodel, deploy_file, use_gpu=True):
    """
    Returns an instance of caffe.Net

    Arguments:
    caffemodel -- path to a .caffemodel file
    deploy_file -- path to a .prototxt file

    Keyword arguments:
    use_gpu -- if True, use the GPU for inference
    """
    if use_gpu:
        caffe.set_mode_gpu()
        caffe.set_device(GPU_DEV)

    # load a new model
    return caffe.Net(deploy_file, caffe.TEST, weights = caffemodel)


def extractitem(net, mean_pixel, fname):
  
    DATA_LAYER = net.inputs[0]
    net.blobs[DATA_LAYER].reshape(1,3,224,224) 
    try:
       input_image = Image.fromarray(np.uint8(caffe.io.load_image(fname)*255))#.resize((256,256),Image.BILINEAR).crop((16,16,240,240))
       #resize
       sf = 256.0/max(input_image.size)
       input_image = input_image.resize((int(input_image.width*sf),int(input_image.height*sf)),Image.BILINEAR)
       pw = (256 - input_image.width)/2
       ph = (256 - input_image.height)/2
       input_image = np.array(input_image)
         
       #make sure image is RGB format
       if input_image.ndim == 2:
         input_image = input_image[...,None]
         input_image = np.repeat(input_image,3, axis=2)
          
       #pad to make 256x256
       input_image = np.pad(input_image,((ph,ph),(pw,pw),(0,0)), mode='edge')
         
       #crop
       input_image = input_image[16:240,16:240]
       
       sys.stdout.flush()
       transformed_image = np.array(input_image,dtype=np.float32)[:,:,::-1].transpose(2,0,1) - mean_pixel
       sys.stdout.flush()
       net.blobs[DATA_LAYER].data[...] = transformed_image
       sys.stdout.flush()
       _ = net.forward()
       sys.stdout.flush()
       blobname=net.blobs.keys()[-1] #should be feat_p for image and feat_a for sketch
       prediction = net.blobs[blobname].data.squeeze()
    
    
    except Exception as e:
       s=str(e)
       print('WARNING: Image was unusable %s' % fname)
       print(s)
       prediction = np.zeros(LAYER_DIMS).astype(np.float32)
    
    return prediction


if __name__ == "__main__":
    net = get_net(MODEL_WEIGHTS_PATH, MODEL_SPEC_PATH)
    sample_img = 'samples/airplane.png'
    feat = extractitem(net, mean_pixel, sample_img)

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


In [2]:
print(feat)

[ 0.00639245  0.08081876  0.0044873   0.11592958  0.05944462 -0.09307066
 -0.04271397  0.03470999  0.03350986 -0.00476146  0.04817637  0.06547271
 -0.02774744 -0.15141903 -0.04547687 -0.03360204 -0.06439552  0.03062941
 -0.00729392  0.07064831 -0.02200796 -0.12362798  0.05309408 -0.01149576
 -0.06780905 -0.03877553  0.02340425 -0.0122044  -0.05903025  0.00317404
 -0.0718967  -0.01488516  0.00252436 -0.04628495 -0.01590104  0.0697663
 -0.06046835  0.01523258  0.03350831 -0.00362619  0.07315936  0.14648522
  0.08918964  0.05478196 -0.03500324  0.04395775  0.12021302 -0.02986014
 -0.11315144 -0.00332042 -0.02751937  0.04182264  0.02970279  0.02741882
 -0.0347417   0.0117903   0.01313386  0.07706376  0.06420449  0.07436155
 -0.06127318  0.12071315  0.02536178 -0.01322524 -0.03562841 -0.13067216
  0.05410939  0.13563858 -0.02241987 -0.00736585 -0.02518882 -0.13285822
 -0.02602011 -0.02939899  0.06749038 -0.02049954 -0.06962994  0.06108927
 -0.06991279 -0.07486835  0.0645859  -0.03202413 -0.